# Scrolling through the history of the Dutch government

After the longest formation period ever (approximately 300 days) the Netherlands has a government once again. Sworn in on January 10th, the fourth Rutte cabinet features several new positions (like a minister for climate and energy), a fair amount of women (14 out of 29) and several very young members. 

Without getting into the specifics of their politics I wanna address that the 50% women truly got me excited. It got me wondering about the long line of white, middle-aged male cabinet members that these new members are joining. Since I am trying to learn more about web scraping and data visualisation I decided to try and create a dataset of all Dutch members of cabinet ever (or since 1860). 

You can read the accompanying blogpost here: and download the complete dataset from: https://almaliezenga.com/2022/02/01/cabinet-members-of-the-netherlands-a-dataset/ 

## Prerequisites 

In [1]:
from lxml import html
import requests
import re
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
import openpyxl 

## The base list of all cabinets

I retrieved most of the data from Wikipedia. The English Wikipedia turned out to have the most complete and machine-readable data on this subject. However, the last two cabinets had a differently formatted or Dutch Wikipedia page and I still wanted to add some additional data on age and gender, so I had to do some extra (manual) work.

I used this page as a base list of all cabinets: https://en.wikipedia.org/wiki/List_of_cabinets_of_the_Netherlands (since 1877). 

This post: https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451 helped me a lot with getting the wikipedia table into a dataframe!

In [2]:
#retrieveing the data from the table in the url 
cabinets_url = 'https://en.wikipedia.org/wiki/List_of_cabinets_of_the_Netherlands' 
table_class = 'wikitable sortable jquery-tablesorter'
response = requests.get(cabinets_url)
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find('table',{'class':"wikitable"})

In [3]:
#building the dataframe from the table 
cabinets_overview = pd.read_html(str(indiatable))
cabinets_overview = pd.DataFrame(cabinets_overview[0])

In [4]:
#data cleaning and reformatiing
cabinets_overview = cabinets_overview.drop(["Term of office.1", "Demissionary", "Time in office", "Legislature Status", "Type", "Election", "Parties", "Orientation"], axis=1)
cabinets_overview['Cabinet'] = cabinets_overview['Cabinet'].str.split("[")
cabinets_overview['Cabinet'] = cabinets_overview['Cabinet'].str[0]
cabinets_overview['Cabinet'] = cabinets_overview['Cabinet'].str.split("(")
cabinets_overview['Cabinet'] = cabinets_overview['Cabinet'].str[0]
cabinets_overview['Cabinet'] = cabinets_overview['Cabinet'].str[:-1]
cabinets_overview = cabinets_overview.drop_duplicates()
#this one was not working out, it was the only one without a space at the end and for some reason they used a 'l' i.o. 'I' in the name... idk
cabinets_overview['Cabinet'][58] = 'Rutte IV'
cabinets_overview.tail(10)

,Cabinet,Prime Minister,Term of office,Political position
49,Kok I,Wim Kok,22 August 1994,Centrist
50,Kok II,Wim Kok,3 August 1998,Centrist
51,Balkenende I,Jan Peter Balkenende,22 June 2002,Right-wing
52,Balkenende II,Jan Peter Balkenende,27 May 2003,Centre-right
53,Balkenende III,Jan Peter Balkenende,7 July 2006,Centre-right
54,Balkenende IV,Jan Peter Balkenende,22 February 2007,Centrist
55,Rutte I,Mark Rutte,14 October 2010,Right-wing
56,Rutte II,Mark Rutte,5 November 2012,Centrist
57,Rutte III,Mark Rutte,26 October 2017,Centre-right
58,Rutte IV,Mark Rutte,10 januari 2022,Centre-right


To actually get the members of the cabinet I also needed the link to the dedicated Wikipedia page for each cabinet. For this, I retrieved all of the links in the Wikipedia page and marged them with the cabinets dataframe. 

In [5]:
cabinets_links = {}

#iterating over all the links on the website 
for link in soup.find_all("a"):
    url = link.get("href", "")
    #adding them to my list if the link contains _cabinet and /wiki/
    if "_cabinet" in url and "/wiki/" in url:
        cabinets_links[link.text.strip()] = url

#data cleaning and reformatiing
cabinets_links = pd.DataFrame.from_dict(cabinets_links, orient='index')
cabinets_links = cabinets_links.reset_index()
cabinets_links = cabinets_links.rename(columns = {'index': 'Cabinet', 0: 'Link'})
cabinets_links['Cabinet'] = cabinets_links['Cabinet'].replace("  ", " ")
cabinets_links.tail(10)

,Cabinet,Link
70,Ruijs de Beerenbrouck II,/wiki/Second_Ruijs_de_Beerenbrouck_cabinet
71,Ruijs de Beerenbrouck III,/wiki/Third_Ruijs_de_Beerenbrouck_cabinet
72,Rutte IV,/wiki/Fourth_Rutte_cabinet
73,Lists of national cabinets,/wiki/Category:Lists_of_national_cabinets
74,Article,/wiki/List_of_cabinets_of_the_Netherlands
75,Talk,/wiki/Talk:List_of_cabinets_of_the_Netherlands
76,Read,/wiki/List_of_cabinets_of_the_Netherlands
77,What links here,/wiki/Special:WhatLinksHere/List_of_cabinets_o...
78,Related changes,/wiki/Special:RecentChangesLinked/List_of_cabi...
79,Simple English,https://simple.wikipedia.org/wiki/List_of_Dutc...


In [6]:
#merging this onto the cabinets overview
cabinets = pd.merge(left=cabinets_overview, right=cabinets_links, how='left', on='Cabinet')
cabinets.tail(10)

,Cabinet,Prime Minister,Term of office,Political position,Link
45,Kok I,Wim Kok,22 August 1994,Centrist,/wiki/First_Kok_cabinet
46,Kok II,Wim Kok,3 August 1998,Centrist,/wiki/Second_Kok_cabinet
47,Balkenende I,Jan Peter Balkenende,22 June 2002,Right-wing,/wiki/First_Balkenende_cabinet
48,Balkenende II,Jan Peter Balkenende,27 May 2003,Centre-right,/wiki/Second_Balkenende_cabinet
49,Balkenende III,Jan Peter Balkenende,7 July 2006,Centre-right,/wiki/Third_Balkenende_cabinet
50,Balkenende IV,Jan Peter Balkenende,22 February 2007,Centrist,/wiki/Fourth_Balkenende_cabinet
51,Rutte I,Mark Rutte,14 October 2010,Right-wing,/wiki/First_Rutte_cabinet
52,Rutte II,Mark Rutte,5 November 2012,Centrist,/wiki/Second_Rutte_cabinet
53,Rutte III,Mark Rutte,26 October 2017,Centre-right,/wiki/Third_Rutte_cabinet
54,Rutte IV,Mark Rutte,10 januari 2022,Centre-right,/wiki/Fourth_Rutte_cabinet


## Getting the cabinet members per cabinet 
Now that we have all the important information on the cabinets and a link to their dedicated page we can gather the members of each cabinet from that dedicated page. 

In [7]:
base_url = 'https://en.wikipedia.org'
counter = 1

#create an empty dataframe
cabinet_members = pd.DataFrame()

#iterate over the cabinets dataframe 
for index, row in cabinets.head(53).iterrows():
    #retrieve the table of cabinet members from the wikipedia page 
    url = base_url + cabinets.loc[index, 'Link']
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    indiatable = soup.find('table',{'class':"wikitable"})
    cabinet_members_1 = pd.read_html(str(indiatable))
    cabinet_members_1 = pd.DataFrame(cabinet_members_1[0])
    print(f'Retrieving: ' + url)
    
    #iterate over all the rows in this table to add their values to our dataframe 
    for i, r in cabinet_members_1.iterrows():
        if cabinet_members_1.loc[i, 'Ministers.2'] != 'Ministers without portfolio':
            #write member to cabinet_members dataframe 
            cabinet_members.loc[counter, 'Cabinet'] = cabinets.loc[index, 'Cabinet']
            cabinet_members.loc[counter, 'Term of office'] = cabinets.loc[index, 'Term of office']
            cabinet_members.loc[counter, 'Political position'] = cabinets.loc[index, 'Political position']
            
            #name of the person 
            try:
                cabinet_members.loc[counter, 'Name'] = cabinet_members_1.loc[i, 'Ministers.2']
            except:
                cabinet_members.loc[counter, 'Name'] = cabinet_members_1.loc[i, ('Minister','Minister.2')]
                
            #title + portofolio 
            try:
                cabinet_members.loc[counter, 'Position'] = str(cabinet_members_1.loc[i, 'Title/Ministry']) + ' of ' + str(cabinet_members_1.loc[i, 'Title/Ministry.1'])
            except:
                try:
                    cabinet_members.loc[counter, 'Position'] = str(cabinet_members_1.loc[i, 'Title/Ministry/Portfolio(s)']) + ' of ' + str(cabinet_members_1.loc[i, 'Title/Ministry/Portfolio(s).1'])                
                except:
                    cabinet_members.loc[counter, 'Position'] = str(cabinet_members_1.loc[i, 'Title/Ministry/Portfolio']) + ' of ' + str(cabinet_members_1.loc[i, 'Title/Ministry/Portfolio.1'])                                        
            #party 
            cabinet_members.loc[counter, 'Party'] = cabinet_members_1.loc[i, 'Party']   
            counter += 1 

Retrieving: https://en.wikipedia.org/wiki/Kappeyne_van_de_Coppello_cabinet
Retrieving: https://en.wikipedia.org/wiki/Van_Lynden_van_Sandenburg_cabinet
Retrieving: https://en.wikipedia.org/wiki/Jan_Heemskerk_cabinet
Retrieving: https://en.wikipedia.org/wiki/Mackay_cabinet
Retrieving: https://en.wikipedia.org/wiki/Van_Tienhoven_cabinet
Retrieving: https://en.wikipedia.org/wiki/R%C3%B6ell_cabinet
Retrieving: https://en.wikipedia.org/wiki/Pierson_cabinet
Retrieving: https://en.wikipedia.org/wiki/Kuyper_cabinet
Retrieving: https://en.wikipedia.org/wiki/De_Meester_cabinet
Retrieving: https://en.wikipedia.org/wiki/Theo_Heemskerk_cabinet
Retrieving: https://en.wikipedia.org/wiki/Cort_van_der_Linden_cabinet
Retrieving: https://en.wikipedia.org/wiki/First_Ruijs_de_Beerenbrouck_cabinet
Retrieving: https://en.wikipedia.org/wiki/Second_Ruijs_de_Beerenbrouck_cabinet
Retrieving: https://en.wikipedia.org/wiki/First_Colijn_cabinet
Retrieving: https://en.wikipedia.org/wiki/First_De_Geer_cabinet
Retrievi

The pages for the third and fourth Rutte cabinet were formatted differently and therefore I wrote some separate code to extract the data for these cabinets. 

In [8]:
#function to slightly make the third and fourth Rutte cabinet easier 
def write_info_R(index_R, i, cabinets, cabinet_members_Rutte, counter): 
    cabinet_members.loc[counter, 'Cabinet'] = cabinets.loc[index_R, 'Cabinet']
    cabinet_members.loc[counter, 'Term of office'] = cabinets.loc[index_R, 'Term of office']
    cabinet_members.loc[counter, 'Political position'] = cabinets.loc[index_R, 'Political position']
    cabinet_members.loc[counter, 'Position'] = cabinet_members_Rutte.loc[i, ('Title', 'Title')]
    cabinet_members.loc[counter, 'Party'] = cabinet_members_Rutte.loc[i, ('Party', 'Party')]   

In [9]:
#Third Rutte cabinet 
index_R3 = 53 
url = base_url + cabinets.loc[index_R3, 'Link']
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find_all('table',{'class':"wikitable"})

cabinet_members_Rutte3 = pd.read_html(str(indiatable))
cabinet_members_Rutte3_0 = pd.DataFrame(cabinet_members_Rutte3[0])
cabinet_members_Rutte3_1 = pd.DataFrame(cabinet_members_Rutte3[1])
cabinet_members_Rutte3_2 = pd.DataFrame(cabinet_members_Rutte3[2])

counter = 1418

for i, r in cabinet_members_Rutte3_0.iterrows():
    #write member to cabinet_members dataframe 
    write_info_R(index_R3, i, cabinets, cabinet_members_Rutte3_0, counter)
    cabinet_members.loc[counter, 'Name'] = cabinet_members_Rutte3_0.loc[i, ('Minister', 'Minister.2')]       
    counter += 1 
    
for i, r in cabinet_members_Rutte3_1.iterrows():
    #write member to cabinet_members dataframe 
    write_info_R(index_R3, i, cabinets, cabinet_members_Rutte3_1, counter)
    cabinet_members.loc[counter, 'Name'] = cabinet_members_Rutte3_1.loc[i, ('Minister', 'Minister.2')]       
    counter += 1 

    #staatssecretarissen is nog even ingewikkeld 
for i, r in cabinet_members_Rutte3_2.iterrows():
    #write member to cabinet_members dataframe 
    write_info_R(index_R3, i, cabinets, cabinet_members_Rutte3_2, counter)
    cabinet_members.loc[counter, 'Name'] = cabinet_members_Rutte3_2.loc[i, ('State secretary', 'State secretary.2')]       
    counter += 1 

In [10]:
#Fourth Rutte cabinet 
index_R4 = 54
url = base_url + cabinets.loc[index_R4, 'Link']
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find_all('table',{'class':"wikitable"})

cabinet_members_Rutte4 = pd.read_html(str(indiatable))
cabinet_members_Rutte4_1 = pd.DataFrame(cabinet_members_Rutte4[1])
cabinet_members_Rutte4_2 = pd.DataFrame(cabinet_members_Rutte4[2])
cabinet_members_Rutte4_3 = pd.DataFrame(cabinet_members_Rutte4[3])

counter = 1472

for i, r in cabinet_members_Rutte4_1.iterrows():
    #write member to cabinet_members dataframe 
    write_info_R(index_R4, i, cabinets, cabinet_members_Rutte4_1, counter)
    cabinet_members.loc[counter, 'Name'] = cabinet_members_Rutte4_1.loc[i, ('Minister', 'Minister.2')]       
    counter += 1 
    
for i, r in cabinet_members_Rutte4_2.iterrows():
    #write member to cabinet_members dataframe 
    write_info_R(index_R4, i, cabinets, cabinet_members_Rutte4_2, counter)
    cabinet_members.loc[counter, 'Name'] = cabinet_members_Rutte4_2.loc[i, ('Minister', 'Minister.2')]       
    counter += 1 

    #staatssecretarissen is nog even ingewikkeld 
for i, r in cabinet_members_Rutte4_3.iterrows():
    #write member to cabinet_members dataframe 
    write_info_R(index_R4, i, cabinets, cabinet_members_Rutte4_3, counter)
    cabinet_members.loc[counter, 'Name'] = cabinet_members_Rutte4_3.loc[i, ('State secretary', 'State secretary.2')]       
    counter += 1 

## Data cleaning

In [11]:
#remove  [Title] and [vii] etc from the positions
cabinet_members['Position'] = cabinet_members['Position'].str.replace(r'\[.*?\]','')

<ipython-input-11-bd0c3ffab1d9>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cabinet_members['Position'] = cabinet_members['Position'].str.replace(r'\[.*?\]','')


In [12]:
#remove some mistaken columns 
wrong_names = ['Source: (in Dutch) Parlement & Politiek', 'Source: (in Dutch) Rijksoverheid.nl', 'Source: (in Dutch) Rijksoverheid', 'Minister without portfolio', 'State Secretaries']

for index, row in cabinet_members.iterrows():
    if row['Name'] in wrong_names:
        cabinet_members = cabinet_members.drop(index)

## Imputing age values

What we have now is a dataframe with all the members of cabinet for the Netherlands since 1877. I want to add the columns age and gender to this dataset. This is what we're gonna do now but first I wanna save the data to make sure I don't have to rerun everything if I make a mistake. 

In [13]:
cabinet_members.to_csv('data/cabinet_members.csv', header=True, index=False)

In [14]:
cabinet_members = pd.read_csv('data/cabinet_members.csv')

In [15]:
#split the name column on '(' signifying the birth years being given 
new = cabinet_members["Name"].str.split("(", n = 1, expand = True)
cabinet_members["Name"]= new[0]

for index, row in cabinet_members.iterrows():
    naam = row['Name']
    if naam[-1] == " ": 
        cabinet_members.loc[index, 'Name'] = naam[:-1]

In [16]:
#creating a new column of the birth year based on the split
cabinet_members["Birth year"] = new[1]
for index, row in cabinet_members.iterrows():
    if 'born' in str(cabinet_members.loc[index, 'Birth year']):
        cabinet_members.loc[index, 'Birth year'] = str(row['Birth year'])[-5:-1]
    else:
        cabinet_members.loc[index, 'Birth year'] = str(row['Birth year'])[0:4]

In [17]:
#add the birth year for Rutte 4
cabinet_members.loc[cabinet_members.Name == 'Mark Rutte', 'Birth year'] = 1967
cabinet_members.loc[cabinet_members.Name == 'Sigrid Kaag', 'Birth year'] = 1961
cabinet_members.loc[cabinet_members.Name == 'Wopke Hoekstra', 'Birth year'] = 1975
cabinet_members.loc[cabinet_members.Name == 'Carola Schouten', 'Birth year'] = 1977
cabinet_members.loc[cabinet_members.Name == 'Dilan Yeşilgöz-Zegerius', 'Birth year'] = 1977
cabinet_members.loc[cabinet_members.Name == 'Hanke Bruins Slot', 'Birth year'] = 1977
cabinet_members.loc[cabinet_members.Name == 'Robbert Dijkgraaf', 'Birth year'] = 1960
cabinet_members.loc[cabinet_members.Name == 'Kajsa Ollongren', 'Birth year'] = 1967
cabinet_members.loc[cabinet_members.Name == 'Mark Harbers', 'Birth year'] = 1969
cabinet_members.loc[cabinet_members.Name == 'Micky Adriaansens', 'Birth year'] = 1964
cabinet_members.loc[cabinet_members.Name == 'Henk Staghouwer', 'Birth year'] = 1962
cabinet_members.loc[cabinet_members.Name == 'Karien van Gennip', 'Birth year'] = 1968
cabinet_members.loc[cabinet_members.Name == 'Ernst Kuipers', 'Birth year'] = 1959
cabinet_members.loc[cabinet_members.Name == 'Liesje Schreinemacher', 'Birth year'] = 1983
cabinet_members.loc[cabinet_members.Name == 'Franc Weerwind', 'Birth year'] = 1964
cabinet_members.loc[cabinet_members.Name == 'Hugo de Jonge', 'Birth year'] = 1977
cabinet_members.loc[cabinet_members.Name == 'Dennis Wiersma', 'Birth year'] = 1986
cabinet_members.loc[cabinet_members.Name == 'Rob Jetten', 'Birth year'] = 1987
cabinet_members.loc[cabinet_members.Name == 'Christianne van der Wal', 'Birth year'] = 1973
cabinet_members.loc[cabinet_members.Name == 'Conny Helder', 'Birth year'] = 1958
cabinet_members.loc[cabinet_members.Name == 'Eric van der Burg', 'Birth year'] = 1965
cabinet_members.loc[cabinet_members.Name == 'Alexandra van Huffelen', 'Birth year'] = 1968
cabinet_members.loc[cabinet_members.Name == 'Gunay Uslu', 'Birth year'] = 1972
cabinet_members.loc[cabinet_members.Name == 'Marnix van Rij', 'Birth year'] = 1960
cabinet_members.loc[cabinet_members.Name == 'Aukje de Vries', 'Birth year'] = 1964
cabinet_members.loc[cabinet_members.Name == 'Christophe van der Maat', 'Birth year'] = 1980
cabinet_members.loc[cabinet_members.Name == 'Vivianne Heijnen', 'Birth year'] = 1982
cabinet_members.loc[cabinet_members.Name == 'Hans Vijlbrief', 'Birth year'] = 1963
cabinet_members.loc[cabinet_members.Name == 'Maarten van Ooijen', 'Birth year'] = 1990  

#apparently Rutte 3 too, this is great 
cabinet_members.loc[cabinet_members.Name == 'Wouter Koolmees', 'Birth year'] = 1977  
cabinet_members.loc[cabinet_members.Name == 'Halbe Zijlstra', 'Birth year'] = 1969 
cabinet_members.loc[cabinet_members.Name == 'Stef Blok', 'Birth year'] = 1964 
cabinet_members.loc[cabinet_members.Name == 'Tom de Bruijn', 'Birth year'] = 1948  
cabinet_members.loc[cabinet_members.Name == 'Ben Knapen', 'Birth year'] = 1951
cabinet_members.loc[cabinet_members.Name == 'Ferd Grapperhaus', 'Birth year'] = 1959
cabinet_members.loc[cabinet_members.Name == 'Raymond Knops', 'Birth year'] = 1971 
cabinet_members.loc[cabinet_members.Name == 'Ingrid van Engelshoven', 'Birth year'] = 1966 
cabinet_members.loc[cabinet_members.Name == 'Ank Bijleveld', 'Birth year'] = 1962
cabinet_members.loc[cabinet_members.Name == 'Henk Kamp', 'Birth year'] = 1952
cabinet_members.loc[cabinet_members.Name == 'Cora van Nieuwenhuizen', 'Birth year'] = 1963
cabinet_members.loc[cabinet_members.Name == 'Barbara Visser', 'Birth year'] = 1962
cabinet_members.loc[cabinet_members.Name == 'Ank Bijleveld', 'Birth year'] = 1977
cabinet_members.loc[cabinet_members.Name == 'Eric Wiebes', 'Birth year'] = 1963
cabinet_members.loc[cabinet_members.Name == """Bas van 't Wout""", 'Birth year'] = 1977
cabinet_members.loc[cabinet_members.Name == 'Sander Dekker', 'Birth year'] = 1975
cabinet_members.loc[cabinet_members.Name == 'Arie Slob', 'Birth year'] = 1961
cabinet_members.loc[cabinet_members.Name == 'Bruno Bruins', 'Birth year'] = 1963
cabinet_members.loc[cabinet_members.Name == 'Martin van Rijn', 'Birth year'] = 1956
cabinet_members.loc[cabinet_members.Name == 'Tamara van Ark', 'Birth year'] = 1974
cabinet_members.loc[cabinet_members.Name == 'Stientje van Veldhoven', 'Birth year'] = 1973
cabinet_members.loc[cabinet_members.Name == 'Ankie Broekers-Knol', 'Birth year'] = 1946
cabinet_members.loc[cabinet_members.Name == 'Menno Snel', 'Birth year'] = 1970
cabinet_members.loc[cabinet_members.Name == 'Steven van Weyenberg', 'Birth year'] = 1973
cabinet_members.loc[cabinet_members.Name == 'Mona Keijzer', 'Birth year'] = 1968
cabinet_members.loc[cabinet_members.Name == 'Paul Blokhuis', 'Birth year'] = 1963

#this one also needed some help 
cabinet_members.loc[cabinet_members['Name'] == 'Hendrikus Colijn', 'Birth year'] = 1869

cabinet_members['Birth year'] = cabinet_members['Birth year'].astype(int)

In [18]:
#reformat the year of taking office
cabinet_members['Year of taking office'] = cabinet_members['Term of office'].str[-4:]
cabinet_members['Year of taking office'] = cabinet_members['Year of taking office'].astype(int)

#calculate the (approximate) age at taking office 
cabinet_members["Age at January 1st"] = cabinet_members['Year of taking office'] - cabinet_members['Birth year']

## Handling multiple positions
I wanted to have the data structured per person, not per position, such that each row is one members of the cabinet and you can see their different positions. This seems more fair especially when you want to use this data to calculate number of women or average age in a cabinet. 

In [19]:
#new empty dataframe 
cabinet_members_2 = pd.DataFrame()

#iterate over cabinets to combine the rows with the same name, positions are seperated by a '; ' 
for cabinet in cabinet_members.Cabinet.unique():
    members = cabinet_members.loc[cabinet_members.Cabinet == cabinet]
    members = members.groupby('Name').agg({'Name':'first', 
                                            'Position': '; '.join, 
                                            'Party':'first', 
                                            'Cabinet': 'first',
                                            'Year of taking office': 'first', 
                                            'Age at January 1st': 'first', 
                                            'Birth year': 'first'})
    cabinet_members_2 = cabinet_members_2.append(members)    
cabinet_members_2 = cabinet_members_2.reset_index(drop=True)
    
#iterate over the new dataframe to split the positions into seperate columns 
for i, r in cabinet_members_2.iterrows():
    positions = cabinet_members_2.loc[i, "Position"].split('; ')
    cabinet_members_2.loc[i, "Position 1"] = positions[0]
    try: 
        cabinet_members_2.loc[i, "Position 2"]= positions[1]
    except: 
        cabinet_members_2.loc[i, "Position 2"]= ""
    try:
        cabinet_members_2.loc[i, "Position 3"]= positions[2]
    except: 
        cabinet_members_2.loc[i, "Position 3"]= ""
    try:
        cabinet_members_2.loc[i, "Position 4"]= positions[3]
    except: 
        cabinet_members_2.loc[i, "Position 4"]= ""

## Adding gender into the mix 

Adding gender to the mix of this data was pretty complicated. Luckily, I found a list of all female cabinet members of the Netherlands. I also was happy for once that I could assume all cabinet members before 1952 were male.  

In [20]:
#retrieving all the tables from the wikipedia page 
url = 'https://en.wikipedia.org/wiki/List_of_female_cabinet_members_of_the_Netherlands' 
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find_all('table',{'class':"wikitable"})
female_cabinet_members_str = pd.read_html(str(indiatable))
female_cabinet_members = pd.DataFrame()

#iterating over the 26 tables on the page and adding the names to my own dataframe 
for i in range(2,27):
    female_cabinet_members_temp = pd.DataFrame(female_cabinet_members_str[i])
    female_cabinet_members = female_cabinet_members.append(female_cabinet_members_temp)
    
female_cabinet_members = female_cabinet_members.reset_index()

#remove the faulty rows 
for index, row in female_cabinet_members.iterrows():
    if 'Source:' in row['Cabinet Member(s).2']:
        female_cabinet_members = female_cabinet_members.drop(index)
    
#we only need the name 
female_cabinet_members = pd.DataFrame(female_cabinet_members['Cabinet Member(s).2'])
female_cabinet_members = female_cabinet_members.rename({'Cabinet Member(s).2': 'Name'}, axis=1)
new = female_cabinet_members['Name'].str.split("(", n = 1, expand = True)
female_cabinet_members['Name']= new[0]
female_cabinet_members['Name'] = female_cabinet_members['Name'].str[:-1]

#adding the gender 
female_cabinet_members['Gender'] = 'Female'

#remove duplicates 
female_cabinet_members = female_cabinet_members.drop_duplicates()

In [21]:
#combine with the cabinet members dataframe 
cabinet_members_3 = pd.merge(left=cabinet_members_2, right=female_cabinet_members, how='left', on='Name')

In [22]:
#manually added the gender of these women because the spelling was slightly different 
cabinet_members_3.loc[cabinet_members_3.Name == 'Anneke van Dok -van Weele', 'Gender'] = 'Female'
cabinet_members_3.loc[cabinet_members_3.Name == 'Gunay Uslu', 'Gender'] = 'Female'
cabinet_members_3.loc[cabinet_members_3.Name == 'Kajsa Ollongren', 'Gender'] = 'Female'
cabinet_members_3.loc[cabinet_members_3.Name == 'Dilan Yeşilgöz-Zegerius', 'Gender'] = 'Female'
cabinet_members_3.loc[cabinet_members_3.Name == 'Ineke Lambers- Hacquebard', 'Gender'] = 'Female'

In [23]:
#setting all the people that took office before 1952 to male 
cabinet_members_3['Year of taking office'] = cabinet_members_3['Year of taking office'].astype(int)

for index, row in cabinet_members_3.iterrows():
    year = row['Year of taking office']
    if year < 1952:
        cabinet_members_3.loc[index, 'Gender'] = 'Male'

In [24]:
#checking if these are indeed all men 
for i, r in cabinet_members_3.iterrows():
    if pd.isnull(cabinet_members_3.loc[i, 'Gender']):
        print(cabinet_members_3.loc[i, 'Name'])

Ad de Bruijn
Dr. Aat van Rhijn
Dr. Gerard Veldkamp
Dr. Jelle Zijlstra
Dr. Julius Christiaan van Oven
Dr. Louis Beel
Dr. Piet Muntendam
Dr. Willem Hendrik van den Berge
Dr. Willem Kernkamp
Ferdinand Kranenburg
Frans-Jozef van Thiel
Herman Witte
Jacob Algera
Jo Cals
Jo van de Kieft
Johan Beyen
Joseph Luns
Kees Staf
Ko Suurhoff
Leendert Antonie Donker
Sicco Mansholt
Vice admiral Harry Moorman
Willem Drees
Dr. Aat van Rhijn
Dr. Anne Vondeling
Dr. Gerard Veldkamp
Dr. Ivo Samkalden
Dr. Jelle Zijlstra
Ernst van der Beugel
Ferdinand Kranenburg
Gerard Helders
Henk Hofstra
Herman Witte
Jacob Algera
Jan van Aartsen
Jo Cals
Joseph Luns
Kees Staf
Ko Suurhoff
Meine van Veen
Norbert Schmelzer
René Höppener
Sicco Mansholt
Teun Struycken
Vice admiral Harry Moorman
Willem Drees
Dr. Gerard Veldkamp
Dr. Jelle Zijlstra
Dr. Louis Beel
Gerard Helders
Herman Witte
Jan van Aartsen
Jo Cals
Joseph Luns
Kees Staf
Norbert Schmelzer
René Höppener
Teun Struycken
Vice admiral Harry Moorman
Albert Beerman
Bauke Roolvi

In [25]:
#set everyone else to male
cabinet_members_3['Gender'] = cabinet_members_3['Gender'].fillna('Male')

## Setting the parties straight
When adding the third and fourth cabinet I noticed that the abbreviations of the parties were suddenly used instead of the full names. I wanted to make this consistent throughout the data and translate some party names to the Dutch name since they are, after all, Dutch parties. 

In [26]:
parties = cabinet_members_3.Party.unique()
parties

array(['Independent Liberal (Classical Liberal)',
       'Independent Liberal (Conservative Liberal)',
       'Independent Conservative (Liberal Conservative)',
       'Independent Christian Democrat (Protestant)',
       'Independent Christian Democrat (Conservative Catholic)',
       'Anti-Revolutionary Party',
       'Independent Christian Democrat (Catholic)', 'Liberal Union',
       'Independent Liberal (Social Liberal)',
       'Free-thinking Democratic League',
       'General League of Roman Catholic Caucuses', 'Independent',
       'Christian Historical Union', 'Roman Catholic State Party',
       'Independent Conservative (Protestant)', 'Liberal State Party',
       "Social Democratic Workers' Party",
       'Independent Classical Liberal',
       'Independent Conservative Liberal', 'Independent Social Democrat',
       'Independent Social Liberal', 'Independent Liberal Conservative',
       'Independent Christian Democratic Protestant',
       "Catholic People's Party", 'Lab

In [28]:
cabinet_members_3["Party"].replace({"Ind.[vi]": 'Independent', 
                                 "Anti-Revolutionary Party": "Anti-Revolutionaire Partij", 
                                 "Liberal Union": "Liberale Unie", 
                                 "General League of Roman Catholic Caucuses": "Algemeene Bond van Roomsch-Katholieke Kiesverenigingen", 
                                 "Free-thinking Democratic League": "Vrijzinnig Democratische Bond", 
                                 "Economic League": "Economische Bond",
                                 "Christian Historical Union": "Christelijk-Historische Unie", 
                                 "Roman Catholic State Party": "Roomsch-Katholieke Staatspartij", 
                                 "Liberal State Party": "Liberale Staatspartij", 
                                 "Social Democratic Workers' Party": "Sociaal-Democratische Arbeiderspartij", 
                                 "Catholic People's Party": "Katholieke Volkspartij",
                                 "Labour Party": "PvdA",
                                 "People's Party for Freedom and Democracy": "VVD",
                                 "Democratic Socialists '70": "Democratisch Socialisten '70",
                                 "Democrats 66": "D66",
                                 "Political Party of Radicals": "PPR",
                                 "Christian Democratic Appeal": "CDA",
                                 "Pim Fortuyn List": "Lijst Pim Fortuyn",
                                 "Christian Union": "CU",
                                }, inplace=True)

In [29]:
#reordering the columns one last time 
cabinet_members_4 = cabinet_members_3[['Name', 'Cabinet', 'Year of taking office', 'Position 1', 'Position 2', 'Position 3', 'Position 4', 'Gender', 'Birth year', 'Age at January 1st', 'Party']]

In [34]:
cabinet_members_4.to_csv('data/cabinet_members_final.csv', header=True, index=False)

In [35]:
cabinet_members_4.to_excel('data/cabinet_members_final.xlsx')

In [36]:
cabinets.to_csv('data/cabinets.csv', header=True, index=False)

In [37]:
cabinets.to_excel('data/cabinets.xlsx', header=True, index=False)